#LangSmithWithLangChain

In [ ]:
!pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
# !pip install -U langsmith
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="<your-api-key>"
# LANGCHAIN_PROJECT="pr-passionate-publisher-27"
#from langchain_openai import ChatOpenAI
# import openai
# from langsmith.wrappers import wrap_openai
# from langsmith import traceable

# # Auto-trace LLM calls in-context
# client = wrap_openai(openai.Client())

@traceable # Auto-trace this function
# def pipeline(user_input: str):
#     result = client.chat.completions.create(
#         messages=[{"role": "user", "content": user_input}],
#         model="gpt-3.5-turbo"
#     )
#     return result.choices[0].message.content

# pipeline("Hello, world!")
# # Out:  Hello there! How can I assist you today?

##Example No 1, Langsmith


In [ ]:
# Cell 1: Install required packages
!pip install langchain langchain-community langchain-core python-dotenv faiss-cpu sentence-transformers pypdf langsmith langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
# Cell 2: Import necessary libraries
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.smith import RunEvalConfig, run_on_dataset
from langsmith import Client, traceable
from langchain_core.runnables import chain
from langchain.smith.evaluation.config import RunEvalConfig
from langchain_groq import ChatGroq
from langchain_core.caches import BaseCache
from langchain_core import callbacks
from langchain.cache import InMemoryCache

In [ ]:
from google.colab import userdata
# Cell 3: Configure environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_API_KEY')  # Replace with your key
os.environ["LANGCHAIN_ENDPOINT"]=userdata.get('LANGSMITH_ENDPOINT')
os.environ["LANGCHAIN_PROJECT"] = "rag-chatbot-evaluation"
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')  # Replace with your key

In [ ]:
# Initialize LangSmith client
client = Client()
# Cell 4: Define PDF path and load document
PDF_PATH = "/content/testPDF.pdf"

In [ ]:
@traceable(name="load_pdf")
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

In [ ]:
# Load the documents
documents = load_pdf(PDF_PATH)
print(f"Number of pages loaded: {len(documents)}")

Number of pages loaded: 1


In [ ]:
# Cell 5: Split documents into chunks
@traceable(name="split_documents")
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

In [ ]:
chunks = split_documents(documents)
print(f"Number of chunks created: {len(chunks)}")
# Inspect first chunk
print("\nFirst chunk content:")
print(chunks[0].page_content[:200])  # First 200 characters

Number of chunks created: 2

First chunk content:
Combine work of multiple collaborators
Understand changes
Support incremental development
Compare and revert to earlier versions
Backup
Parallel versions
Document development (for other developers and


In [ ]:
# Cell 6: Create embeddings and vector store
@traceable(name="create_vectorstore")
def create_vectorstore(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore

In [ ]:
vectorstore = create_vectorstore(chunks)
print("Vector store created successfully")

<ipython-input-10-030c90b8dd4a>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully


In [ ]:
# Test similarity search
test_search = vectorstore.similarity_search("test query", k=1)
print("\nSample similarity search result:")
print(test_search[0].page_content[:200])


Sample similarity search result:
functionality. The aim is to familiarise you with the principles of version control, some good practices, and
get you started on the practical matters.
We're going to walk you through an example. The 


In [ ]:
# Cell 7: Set up the LLM and prompt
@traceable(name="setup_llm")
def setup_llm():
    return ChatGroq(
        model_name="llama-3.3-70b-versatile",
        temperature=0.3,
        groq_api_key=os.environ["GROQ_API_KEY"]
    )

llm = setup_llm()

In [ ]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a7ccc9124a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a7ccc913640>, model_name='llama-3.3-70b-versatile', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# fetching prompt from langsmith profile
from langchain import hub
prompt = hub.pull("rag-chatbot-evaluation-sys-p1/rag-chatbot-evaluation-sys-p1")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rag-chatbot-evaluation-sys-p1', 'lc_hub_repo': 'rag-chatbot-evaluation-sys-p1', 'lc_hub_commit_hash': 'c767114a264d04de21bf4e27055b32c7bda74718a3710b42ffaf0fcd0eaa765e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [ ]:
# Create prompt template
template = dict(dict(prompt[0])['prompt'])['template']

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:")

In [ ]:
# Cell 8: Create RAG chain
@traceable(name="create_rag_chain")
def create_rag_chain(llm, vectorstore, prompt):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    return qa_chain

In [ ]:
qa_chain = create_rag_chain(llm, vectorstore, QA_CHAIN_PROMPT)
qa_chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a7ccc9124a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a7ccc913640>, model_name='llama-3.3-70b-versatile', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), return_source_documents=True, retriev

In [ ]:
# Cell 9: Test the chain with a sample question
test_question = "What is GIT?"
response = qa_chain({"query": test_question})
print("Sample question:", test_question)
print("\nAnswer:", response["result"])

Sample question: What is GIT?

Answer: Git is a distributed version control system (VCS) whose primary user interface is the Unix command line. It keeps a database of the files you put under version control and provides commands to access and update that database.


In [ ]:
print("\nSource Documents Used:")
for i, doc in enumerate(response["source_documents"], 1):
    print(f"\nDocument {i}:")
    print(doc.page_content[:200])


Source Documents Used:

Document 1:
functionality. The aim is to familiarise you with the principles of version control, some good practices, and
get you started on the practical matters.
We're going to walk you through an example. The 

Document 2:
Combine work of multiple collaborators
Understand changes
Support incremental development
Compare and revert to earlier versions
Backup
Parallel versions
Document development (for other developers and


In [ ]:
# Cell 12: Run evaluation
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

# Define the application logic you want to evaluate inside a target function
# The SDK will automatically send the inputs from the dataset to your target function
@traceable(name="setup_evaluation")
def target(inputs: dict) -> dict:
    response = qa_chain({"query": inputs["query"]})  # Access qa_chain from the closure
    return {"response": response["result"].strip()}

In [ ]:
instructions = """Evaluate Document Related Answer against Ground Truth for conceptual similarity and classify true or false:
- False: No conceptual match and similarity
- True: Most or full conceptual match and similarity
- Key criteria: Concept should match, not exact wording.
"""
# Define output schema for the LLM judge
class Grade(BaseModel):
    score: bool = Field(description="Boolean that indicates whether the response is accurate relative to the reference answer")

# Define LLM judge that grades the accuracy of the response relative to reference output
def accuracy(outputs: dict, reference_outputs: dict) -> bool:
  groq_client = ChatGroq(
      model_name="llama-3.3-70b-versatile", # Replace with your desired Groq model
      temperature=0.3,
      groq_api_key=os.environ["GROQ_API_KEY"]
  )
  response = groq_client.invoke(
      f"""ground_truth: {reference_outputs["result"]};
      answer: {outputs["response"]}"""
  )
  # print("\n\n--------------",response.content,"------\n\n",type(response.content))
  # Groq API doesn't directly return a boolean score like OpenAI.
  if "accurate" in response.content.lower() or "correct" in response.content.lower():
    score = True
  else:
      score = False

  return score

In [ ]:
# After running the evaluation, a link will be provided to view the results in langsmith
experiment_results = client.evaluate(
    target,  # Pass only the target function
    data = "rag-qa-eval",
    evaluators = [
        accuracy,
        # can add multiple evaluators here
    ],
    experiment_prefix = "rag-chatbot-evaluation",
    max_concurrency = 2,
)
print("Evaluation config created successfully")

View the evaluation results for experiment: 'first-eval-in-langsmith-9f516e6e' at:
https://smith.langchain.com/o/ddf6a084-bde2-400d-b7ec-b0212b5380fb/datasets/f704ed70-bccd-4e17-8efe-34e5b5b2af2f/compare?selectedSessions=ab00bfb9-f775-4e35-b71c-3bd878c512bc




0it [00:00, ?it/s]

Evaluation config created successfully


In [ ]:
# Cell 11: Create evaluation dataset
test_questions = [
    "Why version control??",
    "Give Practical introduction by example?",
    "Graphical user interfaces of GIT",
    "Setting up Git?",
    # Add more relevant questions
]

@traceable(name="create_eval_dataset")
def create_evaluation_dataset(chain: chain, questions: list):
    dataset_name = "rag-qa-eval"

    for question in questions:
        result = chain({"query": question})
        client.create_example(
            inputs={"query": question},
            outputs=result,
            dataset_name=dataset_name
        )

    return dataset_name

In [ ]:
dataset_name = create_evaluation_dataset(qa_chain, test_questions)
print(f"Created evaluation dataset: {dataset_name}")

Created evaluation dataset: rag-qa-eval


In [ ]:
#ChetuSalesPlaybook_Queries

In [ ]:
# Cell 13: Interactive question answering function
@traceable(name="ask_question")
def ask_question(question: str):
    response = qa_chain({"query": question})
    print("Answer:", response["result"])
    print("\nSource Documents:")
    for i, doc in enumerate(response["source_documents"], 1):
        print(f"\nDocument {i} excerpt:")
        print(doc.page_content[:200])
    return response

In [ ]:
# Example usage:
question = "Explain Features of GIT?"
response = ask_question(question)

In [ ]:
# Specify the dataset name
dataset_name = "rag-qa-eval"

# Fetch the evaluation queue
try:
    dataset = client.read_dataset(dataset_name)
    runs = client.list_runs(
        dataset_name=dataset_name,
        execution_order=True,
        error=False,
        _limit=100 # Adjust the limit as needed
    )
    print(f"Found {len(runs)} runs in the '{dataset_name}' dataset.")

    for run in runs:
        print(f"Run ID: {run.id}")
        print(f"Inputs: {run.inputs}")
        print(f"Outputs: {run.outputs}")

        if run.outputs is not None and 'response' in run.outputs:
          print(f"Response: {run.outputs['response']}") # print the generated response

        if run.reference_outputs:
            print(f"Reference Outputs: {run.reference_outputs}")

        print("-" * 20)

except Exception as e:
    print(f"Error accessing the dataset or queue: {e}")

#######Act as a Deep Learning and LLM Expert who have lots of experiance in Langchain Framework, now I need your help to create a program which will be executed in Google Colab so that means the code will be written in an Interprative way, now here we are going to create a Langchain RAG based chatbot where we are going to pass the path of a PDF file in static variable, lanchain will split the text create chunks will use Faiss vactor database to store the vactor information, will use all-LM-L6-V2 model from hugging face for embadding and will use "llama-3.3-70b-versatile" model through Groq API. and then will perform question answering related to the document, now this was all Langchain RAG pipeline part, now the main thing I want to test is Langsmith for proper Debugging, Testing, Evaluating, Monitoring, Usage metrics for the created document chatbot. In which you are going to explain how to do those things through Langsmith UI (I already have the api-key) and how to do all these 5 steps in the colab through code. So as you might understand by now I am trying to create a LLM pipeline through langsmith, so Explain everything to me like I dont know anything about this framework who just have a little knowledge of langchain RAG, Groq and python. and please write the code in an Interprative way like I am writing it on Google Colab.

In [ ]:
#using "from langsmith import traceable" for @traceable decorator, and also think you have imported wrong library for groq this is the right "from langchain_groq import ChatGroq"?